In [ ]:
%pip install "spacy>=3"
%pip install spacy-transformers





In [ ]:
import spacy
spacy.info()
from spacy.cli.project.run import project_run
from spacy.cli.project.assets import project_assets
from pathlib import Path


root = Path("/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp")
project_assets(root)




ℹ Fetching 5 asset(s)
✔ Asset already exists:
/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/assets/osterhout.corpus
✔ Asset already exists:
/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/assets/osterhout.verbs
✔ Asset already exists:
/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/assets/kim.corpus
✔ Asset already exists:
/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/assets/kim.verbs
✔ Downloaded asset
/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/assets/vectors.zip


In [ ]:
!python -m spacy download en_core_web_lg
project_run(root, "preprocess_kim", capture=True, force=True)

In [ ]:
project_run(root, "train", capture=True)



=================================== train ===================================
Running command: mkdir -p training/bert
Running command: /usr/bin/python3 -m spacy train ./configs/bert.cfg -o training/bert --gpu-id -1


In [ ]:
project_run(root, "evaluate", capture=True)



================================== evaluate ==================================
Running command: /usr/bin/python3 -m spacy evaluate ./training/bert/model-best ./corpus/test.spacy --output ./metrics/bert.json


{
  "token_acc":1.0,
  "cats_score":0.9854204477,
  "speed":247.8028288108,
  "cats_f_per_type":{
    "Se":{
      "p":0.8421052632,
      "r":0.8421052632,
      "f":0.8421052632
    },
    "Sy":{
      "p":0.85,
      "r":0.85,
      "f":0.85
    },
    "NA":{
      "p":1.0,
      "r":1.0,
      "f":1.0
    }
  },
  "cats_auc_per_type":{
    "Se":0.9773139746,
    "Sy":0.9789473684,
    "NA":1.0
  }
}

In [ ]:
project_run(root, "train_syn", capture=True)



================================= train_syn =================================
Running command: mkdir -p training/bert_syn
Running command: /usr/bin/python3 -m spacy train ./configs/bert_syn.cfg -o training/bert_syn --gpu-id -1


In [ ]:
project_run(root, "train_sem", capture=True)


================================= train_sem =================================
Running command: mkdir -p training/bert_sem
Running command: /usr/bin/python3 -m spacy train ./configs/bert_sem.cfg -o training/bert_sem --gpu-id -1


In [ ]:
project_run(root, "evaluate_ss_kim", capture=True, force=True)



============================== evaluate_ss_kim ==============================
Running command: /usr/bin/python3 -m spacy evaluate ./training/bert_syn/model-best ./corpus/syn_test.spacy --output ./metrics/bert_syn.json
Running command: /usr/bin/python3 -m spacy evaluate ./training/bert_sem/model-best ./corpus/sem_test.spacy --output ./metrics/bert_sem.json
Running command: /usr/bin/python3 ./scripts/eval_ss_kim.py ./training/bert_sem/model-best ./training/bert_syn/model-best ./corpus/syn_test.spacy ./corpus/sem_test.spacy metrics/kim_ss.json


evaluate jointly

In [ ]:
import spacy
model_sem= "/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/training/bert_sem/model-best"
model_syn = "/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/training/bert_syn/model-best"
test_sem_file = "/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/corpus/sem_test.spacy"
test_syn_file = "/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/corpus/syn_test.spacy"
nlp_sem = spacy.load(model_sem)
nlp_syn = spacy.load(model_syn)


In [ ]:
doc=nlp_syn("Olson's bulging suitcase had been forecasting correctly.")
print(doc.cats)
doc=nlp_sem("Olson's bulging suitcase had been forecasting correctly.")
print(doc.cats)

{'NA': 0.9967882633209229, 'Sy': 0.003612011205404997}
{'NA': 0.007195719517767429, 'Se': 0.9970500469207764}


In [ ]:
from spacy.tokens import DocBin, Doc
from spacy.training import Example
from spacy.scorer import Scorer
docbin_sem = DocBin().from_disk(test_sem_file)
docs_sem = sorted([doc for doc in docbin_sem.get_docs(nlp_sem.vocab)], key = lambda d: d.text)
docbin_syn = DocBin().from_disk(test_syn_file)
docs_syn = sorted([doc for doc in docbin_syn.get_docs(nlp_syn.vocab)], key = lambda d: d.text)


preds = []
golds = []

def get_final_label(syn, sem):
  d = {
       ("NA", "NA"): "NA",
       ("NA", "Se"): "Se",
       ("Sy", "NA"): "Sy",
       ("Sy", "Se"): "SS"
  }
  return d[(syn, sem)]

def get_best_label(d):
  return max(d, key=d.get)

examples = []
for i, (doc_syn, doc_sem) in enumerate(zip(docs_syn, docs_sem)):
  gold_label=get_final_label(get_best_label(doc_syn.cats), get_best_label(doc_sem.cats))
  golds.append(gold_label)
  pred_sem = nlp_sem(doc_sem.text)
  pred_syn = nlp_syn(doc_syn.text)
  pred_label = get_final_label(get_best_label(pred_syn.cats), get_best_label(pred_sem.cats))
  preds.append(pred_label)

  gold = Doc(pred_sem.vocab).from_bytes(pred_sem.to_bytes())
  gold.cats ={"NA": 0, "SS":0, "Se":0, "Sy":0}
  gold.cats[gold_label] = 1
  
  pred = Doc(pred_sem.vocab).from_bytes(pred_sem.to_bytes())
  pred.cats ={"NA": 0, "SS":0, "Se":0, "Sy":0}
  pred.cats[pred_label] = 1
  
  examples.append(Example(pred, gold))

  print(i, "\n syn ", doc_syn, doc_syn.cats, f"pred_syn: {pred_syn.cats}", "\n sem ", doc_sem, doc_sem.cats, f"pred_sem: {pred_sem.cats}")

scorer = Scorer()
scores = scorer.score_cats(examples, "cats", labels = ["NA", "Sy", "Se", "SS"])
import json
print(json.dumps(scores, indent=4))

import numpy as np
from sklearn.metrics import precision_recall_fscore_support
y_true = np.array(golds)
y_pred = np.array(preds)

# score per label
print("prf per label")
print(precision_recall_fscore_support(y_true, y_pred, average=None, labels=['NA', 'Se', 'Sy', 'SS']))
# Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.
print("prf macro")
print(precision_recall_fscore_support(y_true, y_pred, average='macro'))
# Calculate metrics globally by counting the total true positives, false negatives and false positives.
print("prf micro")
print(precision_recall_fscore_support(y_true, y_pred, average='micro'))
# Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.
print("prf weighted")
print(precision_recall_fscore_support(y_true, y_pred, average='weighted'))


0 
 syn  An urgent message had been sending by the ambassador to the President. {'NA': 0, 'Sy': 1} pred_syn: {'NA': 0.00018626819655764848, 'Sy': 0.999649167060852} 
 sem  An urgent message had been sending by the ambassador to the President. {'NA': 1, 'Se': 0} pred_sem: {'NA': 0.9996090531349182, 'Se': 0.0003326638834550977}
1 
 syn  An urgent message had been sent again. {'NA': 1, 'Sy': 0} pred_syn: {'NA': 0.9986979961395264, 'Sy': 0.0012557265581563115} 
 sem  An urgent message had been sent again. {'NA': 1, 'Se': 0} pred_sem: {'NA': 0.999031662940979, 'Se': 0.0008203814504668117}
2 
 syn  Billy's unruly hair was curing his cold. {'NA': 1, 'Sy': 0} pred_syn: {'NA': 0.9992364645004272, 'Sy': 0.0014564942102879286} 
 sem  Billy's unruly hair was curing his cold. {'NA': 0, 'Se': 1} pred_sem: {'NA': 0.9583828449249268, 'Se': 0.0584445558488369}
3 
 syn  Carrie had been canceling all of her meetings. {'NA': 1, 'Sy': 0} pred_syn: {'NA': 0.9986732006072998, 'Sy': 0.0008457086514681578} 
 s

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(y_pred)
print(y_true)

conversion de sem en syn

In [ ]:
test_sem_file = "/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/corpus/sem_test.spacy"
train_sem_file = "/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/corpus/sem_train.spacy"
dev_sem_file = "/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/corpus/sem_dev.spacy"

test_syn_file = "/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/corpus/syn_test.spacy"
train_syn_file = "/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/corpus/syn_train.spacy"
dev_syn_file = "/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/corpus/syn_dev.spacy"

sentences = open("/content/drive/MyDrive/collab/erp_textcat/goe_inspired_erp/assets/corpus.tab").read().strip().split("\n")
sentence_dict = {s.split("#")[1]: s.split("#") for s in sentences}

def convert(input_file, output_file, nlp_sem, sentence_dict):
  docbin_sem = DocBin().from_disk(input_file)
  docs = [doc for doc in docbin_sem.get_docs(nlp_sem.vocab)]

  conversion_dict = {"SS": "Sy", "Sy": "Sy", "Se": "NA", "NA": "NA"}
  categories = set([v for v in conversion_dict.values()])
  for doc in docs:
      _id, text, trigger, effect = sentence_dict[doc.text]
      doc.cats = {category: 0 for category in categories}
      doc.cats[conversion_dict[effect]] = 1

  docbin = DocBin(docs=docs, store_user_data=True)
  docbin.to_disk(output_file)

convert(test_sem_file, test_syn_file, nlp_sem, sentence_dict)
convert(train_sem_file, train_syn_file, nlp_sem, sentence_dict)
convert(dev_sem_file, dev_syn_file, nlp_sem, sentence_dict)
